In [28]:
import re
import pandas as pd
from collections import Counter
from eunjeon import Mecab
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Embedding, Dense, GRU
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint




# 준비 작업
df=pd.read_excel('한국어_단발성_대화_데이터셋.xlsx')
print(df.columns)
df=df.drop(['Unnamed: 2','Unnamed: 3','Unnamed: 4','공포',5468], axis=1)
print(df.head())
df=df.replace('공포',5)
df=df.replace('분노',5)
df=df.replace('혐오',5)
df=df.replace('슬픔',4)
df=df.replace('놀람',3)
df=df.replace('중립',2)
df=df.replace('행복',1)
df['Emotion']=np.select([df.Emotion<=3],[1],default=0)
print(df.head())
print(df.isnull().values.any())
train_data,test_data=train_test_split(df,test_size=0.25,random_state=42)
print(len(train_data))
print(len(test_data))
print(train_data.groupby('Emotion').size().reset_index(name = 'count'))


# 데이터 정제
train_data['Sentence'] = train_data['Sentence'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
train_data['Sentence'].replace('', np.nan, inplace=True)
print(train_data.isnull().sum())

test_data.drop_duplicates(subset=['Sentence'],inplace=True)
test_data['Sentence']=test_data['Sentence'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
test_data['Sentence'].replace('', np.nan, inplace=True)
test_data = test_data.dropna(how='any') # Null 값 제거
print('전처리 후 테스트용 샘플의 개수 :',len(test_data))

# 토큰화 
mecab=Mecab()
stopwords = ['도', '는', '다', '의', '가', '이', '은', '한', '에', '하', '고', '을', '를', '인', '듯', '과', '와', '네', '들', '듯', '지', '임', '게']
train_data['tokenized'] = train_data['Sentence'].apply(mecab.morphs)
train_data['tokenized'] = train_data['tokenized'].apply(lambda x: [item for item in x if item not in stopwords])

test_data['tokenized'] = test_data['Sentence'].apply(mecab.morphs)
test_data['tokenized'] = test_data['tokenized'].apply(lambda x: [item for item in x if item not in stopwords])

negative_words = np.hstack(train_data[train_data.Emotion == 0]['tokenized'].values)
positive_words = np.hstack(train_data[train_data.Emotion == 1]['tokenized'].values)

# 긍정과 부정의 단어 빈도수 파악하기 
negative_word_count = Counter(negative_words)
print(negative_word_count.most_common(20))

positive_word_count = Counter(positive_words)
print(positive_word_count.most_common(20))

X_train = train_data['tokenized'].values
y_train = train_data['Emotion'].values
X_test= test_data['tokenized'].values
y_test = test_data['Emotion'].values

# 정수 부여, (데이터 들에 )
tokenizer=Tokenizer()
tokenizer.fit_on_texts(X_train)

# 빈도 수 낮은 단어 제거 
threshold = 2
total_cnt = len(tokenizer.word_index) # 단어의 수
rare_cnt = 0 
total_freq = 0 
rare_freq = 0 

# 단어와 빈도수의 쌍(pair)을 key와 value로 받는다.
for key, value in tokenizer.word_counts.items():
    total_freq = total_freq + value

    # 단어의 등장 빈도수가 threshold보다 작으면
    if(value < threshold):
        rare_cnt = rare_cnt + 1
        rare_freq = rare_freq + value

print('단어 집합(vocabulary)의 크기 :',total_cnt)
print('등장 빈도가 %s번 이하인 희귀 단어의 수: %s'%(threshold - 1, rare_cnt))
print("단어 집합에서 희귀 단어의 비율:", (rare_cnt / total_cnt)*100)
print("전체 등장 빈도에서 희귀 단어 등장 빈도 비율:", (rare_freq / total_freq)*100)

vocab_size=total_cnt-rare_cnt+2
tokenizer=Tokenizer(vocab_size, oov_token='OOV')
tokenizer.fit_on_texts(X_train)
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

print(X_train[:3])

print('사람 발화의 최대 길이 :',max(len(l) for l in X_train))
print('사람 발화의 평균 길이 :',sum(map(len, X_train))/len(X_train))
# plt.hist([len(s) for s in X_train], bins=50)
# plt.xlabel('length of samples')
# plt.ylabel('number of samples')
# plt.show()

def below_threshold_len(max_len, nested_list):
    count = 0
    for sentence in nested_list:
        if(len(sentence) <= max_len):
            count = count + 1
    print('전체 샘플 중 길이가 %s 이하인 샘플의 비율: %s'%(max_len, (count / len(nested_list))*100))

max_len=40
below_threshold_len(max_len,X_train)

X_train = pad_sequences(X_train, maxlen = max_len)
X_test = pad_sequences(X_test, maxlen = max_len)

# Gru로 모델 훈련시키기 
embedding_dim = 100
hidden_units = 128

model = Sequential()
model.add(Embedding(vocab_size, embedding_dim))
model.add(GRU(hidden_units))
model.add(Dense(1, activation='sigmoid'))

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4)
mc = ModelCheckpoint('best_model.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)

model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
history = model.fit(X_train, y_train, epochs=15, callbacks=[es, mc], batch_size=64, validation_split=0.2)

loaded_model = load_model('best_model.h5')
print("\n 테스트 정확도: %.4f" % (loaded_model.evaluate(X_test, y_test)[1]))




def baememory_predict(new_sentence):
    new_sentence = re.sub(r'[^ㄱ-ㅎㅏ-ㅣ가-힣 ]','', new_sentence)
    new_sentence = mecab.morphs(new_sentence) # 토큰화
    new_sentence = [word for word in new_sentence if not word in stopwords] # 불용어 제거
    encoded = tokenizer.texts_to_sequences([new_sentence]) # 정수 인코딩
    pad_new = pad_sequences(encoded, maxlen = max_len) # 패딩

    score = float(loaded_model.predict(pad_new)) # 예측
    if(score > 0.5):
        print("{:.2f}% 확률로 나쁜 기억이 아닙니다.".format(score * 100))
    else:
        print("{:.2f}% 확률로 나쁜 기억입니다.".format((1 - score) * 100))
    
badmemory_predict("아 오늘 너무 놀랏어...이건 무서워...  ")

Index(['Sentence', 'Emotion', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4', '공포',
       5468],
      dtype='object')
                   Sentence Emotion
0  언니 동생으로 부르는게 맞는 일인가요..??      공포
1              그냥 내 느낌일뿐겠지?      공포
2            아직너무초기라서 그런거죠?      공포
3             유치원버스 사고 낫다던데      공포
4               근데 원래이런거맞나요      공포
                   Sentence  Emotion
0  언니 동생으로 부르는게 맞는 일인가요..??        0
1              그냥 내 느낌일뿐겠지?        0
2            아직너무초기라서 그런거죠?        0
3             유치원버스 사고 낫다던데        0
4               근데 원래이런거맞나요        0
False
28945
9649
   Emotion  count
0        0  16359
1        1  12586
Sentence    0
Emotion     0
dtype: int64
전처리 후 테스트용 샘플의 개수 : 9641


<ipython-input-28-d01d9bd4772e>:40: FutureWarning: The default value of regex will change from True to False in a future version.
  train_data['Sentence'] = train_data['Sentence'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
<ipython-input-28-d01d9bd4772e>:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data['Sentence'] = train_data['Sentence'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
C:\Users\playg\anaconda3\envs\deeplearn\lib\site-packages\pandas\core\generic.py:6610: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)
C:\Users\playg\anaconda3\e

[('안', 1412), ('있', 1372), ('면', 1304), ('거', 1286), ('나', 1211), ('없', 1078), ('는데', 1054), ('아', 1052), ('되', 957), ('겠', 928), ('만', 893), ('어', 867), ('같', 799), ('보', 799), ('네요', 784), ('것', 738), ('냐', 726), ('말', 711), ('으로', 693), ('로', 681)]
[('있', 1069), ('ㅋㅋㅋ', 1069), ('거', 910), ('ㅋㅋ', 850), ('면', 759), ('보', 713), ('나', 684), ('ㄷ', 596), ('는데', 582), ('좋', 558), ('안', 539), ('만', 527), ('으로', 523), ('잘', 519), ('에서', 515), ('어', 506), ('진짜', 501), ('해', 488), ('되', 479), ('로', 472)]
단어 집합(vocabulary)의 크기 : 22155
등장 빈도가 1번 이하인 희귀 단어의 수: 10883
단어 집합에서 희귀 단어의 비율: 49.12209433536448
전체 등장 빈도에서 희귀 단어 등장 빈도 비율: 4.439413408390952
[[1725, 1, 214, 5192], [265, 589, 63, 641, 993, 12, 66, 15, 18], [346, 411, 23, 13, 9, 97, 4437]]
사람 발화의 최대 길이 : 101
사람 발화의 평균 길이 : 8.46933840041458
전체 샘플 중 길이가 40 이하인 샘플의 비율: 99.93090343755398
Epoch 1/15
361/362 [============================>.] - ETA: 0s - loss: 0.5540 - acc: 0.7068
Epoch 00001: val_acc improved from -inf to 0.73830, saving model to bes